In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import pandas as pd
#from ray import tune
#from ray.tune import grid_search, uniform
from torch.utils.data import DataLoader, TensorDataset
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import argparse
import scipy.io
import os


In [3]:
import boto3
import io
import scipy.io

# Specify the S3 bucket and file path
bucket_name = 'sagemaker-studio-12bcc0mjktib'
file_key = '2LVSI_scaled_extra.mat'

# Create a Boto3 S3 client
s3_client = boto3.client('s3')

# Download the file from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
file_content = response['Body'].read()

# Load the .mat file using scipy.io
mat_data = scipy.io.loadmat(io.BytesIO(file_content))

# Continue with the rest of your code
input_scaled = mat_data.get('input_scaled')
output_scaled = mat_data.get('output_scaled')

# Split the dataset into input and output subsets
input_subset = input_scaled
output_subset = output_scaled[:,1]

# Verify the shape of the resulting subsets
print(input_subset.shape)
print(output_subset.shape)


(719308, 41)
(719308,)


In [4]:
#Zscore Scaling
#scaler = StandardScaler()
#input_data_scaled = scaler.fit_transform(input_data)
#output_data_scaled = scaler.fit_transform(output_data)
#output_data_scaled = output_data_scaled[:,0]
#print(output_data_scaled)

In [5]:
# Set device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_data = torch.Tensor(input_subset)
output_data = torch.Tensor(output_subset)


# Split the data into training, validation, and testing sets
#test_size = 0.2 ist 80% Training set
train_data, test_data, train_target, test_target = train_test_split(
    input_data, output_data, test_size=0.8, random_state=42
)

#test_size hier auf 0.5 heißt die übrigen 20% nochmal in 10% für val und 10% für test
val_data, test_data, val_target, test_target = train_test_split(
    test_data, test_target, test_size=0.2, random_state=42
)
#data = pd.DataFrame(train_data.numpy())
#print(len(data.iloc[:,7]))
#unique_values = data.iloc[:,7].unique()
#print(len(unique_values))
# Create data loaders
train_dataset = TensorDataset(train_data, train_target)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(val_data, val_target)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(test_data, test_target)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

cpu


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Create an SVR model with hyperparameters
svr = SVR(kernel='linear', C=1.0, epsilon=0.1)

# Train the SVR model using your training data
svr.fit(train_data, train_target)

# Make predictions on the validation set
val_predictions = svr.predict(val_data)

# Calculate the R-squared score to evaluate the SVR model on the validation set
val_r2 = r2_score(val_target, val_predictions)
print(f"Validation R-squared Score: {val_r2}")


In [ ]:
'''
#You can optionally perform hyperparameter tuning using GridSearchCV
# Define hyperparameters to search over
param_grid = {
    'C': [0.1, 1.0, 10.0],      # Regularization parameter
    'epsilon': [0.01, 0.1, 1.0]  # Epsilon parameter
}

# Create a grid search object
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=3, scoring='r2')

# Fit the grid search to your training data to find the best hyperparameters
grid_search.fit(train_data, train_target)

# Get the best SVR model with optimal hyperparameters
best_svr = grid_search.best_estimator_

# Train the best SVR model on the training data
best_svr.fit(train_data, train_target)

# Make predictions on the test set using the best SVR model
test_predictions = best_svr.predict(test_data)

# Calculate the R-squared score to evaluate the best SVR model on the test set
test_r2 = r2_score(test_target, test_predictions)
print(f"Test R-squared Score: {test_r2}")
'''